In [1]:
import warnings

warnings.filterwarnings("ignore")

In [2]:
import base64
import io
import json

import cv2 as cv
import numpy as np
from PIL import Image
from tqdm import tqdm


In [ ]:
INPUT_VIDEO = "data/sup.mov"
OUTPUT_JSON = "out/sup.json"

In [ ]:
def read_video(filename: str) -> cv.VideoCapture:
    return cv.VideoCapture(filename)


def get_video_properties(cap: cv.VideoCapture) -> tuple[int, int, int, int]:
    frame_width, frame_height, fps, frame_count = map(
        int,
        map(
            cap.get,
            [
                cv.CAP_PROP_FRAME_WIDTH,
                cv.CAP_PROP_FRAME_HEIGHT,
                cv.CAP_PROP_FPS,
                cv.CAP_PROP_FRAME_COUNT,
            ],
        ),
    )

    return frame_width, frame_height, fps, frame_count

In [ ]:
def process_frame(frame: cv.UMat, kernel: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    frame_rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    frame_rgba = cv.cvtColor(frame_rgb, cv.COLOR_RGB2RGBA)

    lower_black = np.array([0, 0, 0], dtype=np.uint8)
    upper_black = np.array([10, 10, 10], dtype=np.uint8)

    mask = cv.inRange(frame_rgb, lower_black, upper_black)
    mask = cv.GaussianBlur(mask, (5, 5), 0)
    mask = cv.morphologyEx(mask, cv.MORPH_CLOSE, kernel)
    mask = cv.morphologyEx(mask, cv.MORPH_OPEN, kernel)

    frame_rgba[mask == 255] = (255, 255, 255, 0)

    return frame_rgba, mask

In [ ]:
def encode_frame(frame_rgba: np.ndarray) -> str:
    byte_io = io.BytesIO()
    pil_image = Image.fromarray(frame_rgba)
    pil_image.save(byte_io, format="WEBP", lossless=False)
    byte_io.seek(0)

    return base64.b64encode(byte_io.read()).decode("utf-8")

In [ ]:
def save_lottie(
        filename: str,
        w: int,
        h: int,
        fr: int,
        op: int,
        assets: list[dict],
        layers: list[dict],
) -> None:
    with open(filename, "w") as file:
        json.dump(
            {
                "v": "5.5.2",
                "fr": fr,
                "ip": 0,
                "op": op,
                "h": h,
                "w": w,
                "ddd": 0,
                "assets": assets,
                "layers": layers,
            },
            file,
        )

In [ ]:
def main() -> None:
    cap = read_video(INPUT_VIDEO)
    w, h, fr, op = get_video_properties(cap)

    assets = []
    layers = []
    cursor = 0
    kernel = np.ones((5, 5), np.uint8)

    with tqdm(total=op, desc="Processing frames", unit="frame") as pbar:
        while cap.isOpened():
            ret, frame = cap.read()

            if not ret:
                break

            frame_rgba, _ = process_frame(frame, kernel)
            payload = encode_frame(frame_rgba)

            assets.append(
                {
                    "id": str(cursor + 1),
                    "w": w,
                    "h": h,
                    "p": f"data:image/webp;base64,{payload}",
                    "u": "",
                    "e": 1,
                }
            )
            layers.append(
                {
                    "ind": cursor,
                    "ty": 2,
                    "refId": str(cursor + 1),
                    "sr": 1,
                    "ks": {
                        "o": {"a": 0, "k": 100},
                        "r": {"a": 0, "k": 0},
                        "p": {"a": 0, "k": [0, 0, 0]},
                        "a": {"a": 0, "k": [0, 0, 0]},
                        "s": {"a": 0, "k": [100, 100, 100]},
                    },
                    "ao": 0,
                    "ip": cursor,
                    "op": cursor + 1,
                    "st": cursor,
                    "bm": 0,
                }
            )

            cursor += 1
            pbar.update(1)

    cap.release()
    save_lottie(OUTPUT_JSON, w, h, fr, op, assets, layers)

In [ ]:
if __name__ == "__main__":
    main()